# Azira Bulk Dataset Submission

This notebook includes codes to automatically submit GeoJSONs to Azira at https://pinnacle.azira.com/.

The following reports are requested:
- Pin Report
- Expanded Common Evening, Common Daytime Location

It uses the Playwright browser testing framework.

In [72]:
#from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright, expect, TimeoutError
import json
from pathlib import Path
import time
from datetime import date, timedelta

## 1. Run First 
- needed functions and data

In [73]:
# login info 
user, pw = 'katrina.mullan@mso.umt.edu', 'wvp6rau6rqb_bwy1EYT!'
url = 'https://pinnacle.azira.com/'

# start and end dates for data downloads
start_date = '2022-01-01'
end_date = date.today() - timedelta(days=7)
end_date = end_date.strftime("%Y-%m-%d")

async def start_browser():
    playwright = await async_playwright().start()

    browser = await playwright.chromium.launch(headless=False)

    context = await browser.new_context(
        geolocation={"latitude": 46.8721, "longitude": -113.9940},
        permissions=["geolocation"],
    )

    page = await context.new_page()

    return {
        "playwright": playwright,
        "browser": browser,
        "context": context,
        "page": page,
    }

async def login(page, user, pw):
    await page.goto(url) 
    await page.get_by_role("textbox", name="Email Address").fill(user)
    await page.get_by_role("textbox", name="Password").fill(pw)
    await page.get_by_role("button", name="Log in").click()

def splitJson(json_path):
    json_data = json.loads(json_path.read_text())
    polys = json_data['features'][0]['geometry']['coordinates'].copy()
    num_polys = len(polys)

    # split data in half
    json_data_copy = json_data.copy()
    json_data['features'][0]['geometry']['coordinates'] = polys[:num_polys//2]
    json_data_copy['features'][0]['geometry']['coordinates'] = polys[num_polys//2:]

    # write new data
    part_a_path = json_path.parent / f"{json_path.stem}_a.geojson"
    part_b_path = json_path.parent / f"{json_path.stem}_b.geojson"

    part_a_path.write_text(json.dumps(json_data, indent=2))
    part_b_path.write_text(json.dumps(json_data_copy, indent=2))

    return part_a_path, part_b_path


## 2. Bulk Submit

Get list of files to submit

In [ ]:
submit_list = []

base_dir = Path('/home/vince/Documents/SmartFires/osm_fitness')
folders = [p for p in base_dir.iterdir() if p.is_dir()]

# ----REMOVE NEXT LINE TO DO ALL STATES-----
folders = [f for f in folders if 'Oklahoma' in f.name]

for folder in folders:  
    geojson_files = list(folder.glob('*.geojson'))
    for json_path in geojson_files:
        submit_list.append(json_path)  

submit_list = submit_list

Submit

In [ ]:
# login 
browser_handle = await start_browser()
page = browser_handle["page"]
await login(page, user, pw)

completed = []
failed = []

submit_stack = list(submit_list)
first = True

while submit_stack:
    json_file = submit_stack.pop()
    try:
        json_path = Path(json_file)
        json_text = json_file.read_text() # paste the GeoJSON data

        # for use on the last page before submitting the job
        city = json_path.stem.replace(' ', '_')
        state = json_path.parent.stem.split('_')[0].replace(' ', '_')
        project_name = f"{city}_{state}"

        if first:
            time.sleep(3)
            first = False   

        await page.goto('https://pinnacle.azira.com/home/feeds/datasets') # go to main page

        await page.get_by_role("button", name="Create New").click()  # "Create New"
        await page.get_by_role("button", name="Next").click()  

        await page.get_by_role("heading", name="GeoJSON/KML/ESRI").click()   # click "GEoJSON/KML/ESRI"

        await page.evaluate("() => document.querySelector('#GEOJSON')?.focus()")
        await page.evaluate(
                    """(text) => {
                        const el = document.querySelector('#GEOJSON');
                        el.value = text;
                        el.dispatchEvent(new Event('input', { bubbles: true }));
                        el.dispatchEvent(new Event('change', { bubbles: true }));
                    }""",
                    json_text
                )
        #await page.get_by_role("textbox", name="Enter GeoJSON").fill(json_text)
        await page.get_by_role("button", name="Next").click() 

        await page.locator("#start-date-input").fill(start_date)
        await page.locator("#end-date-input").fill(end_date)
        await page.get_by_role("button", name="Next").click()

        await page.get_by_text("Pin Dataset").click() 
        await page.get_by_text("Common Evening and Daytime").click()   
        await page.get_by_role("button", name="Next").first.click() 

        await page.get_by_role("textbox", name="Name is required").fill(project_name)
        await page.get_by_role("button", name="Submit").first.click()

        # continue if "file too large"  warning appears  
        try:
            await expect(page.get_by_text("Expanded access")).to_be_visible(timeout=1000)
            print(f"{json_file}: File too large")     
            
            # split file and add to stack
            for new_file in splitJson(json_path):
                submit_stack.append(new_file)      

            continue
        except Exception as e:
            pass

        print(f"Completed {json_file}") 
        completed.append(json_file)

    except Exception as e:
        print(f"Error processing {json_file}: {e}")     
        failed.append(json_file) 


# Development Code (for testing)
- code written in developing the above workflow

In [18]:
# login info 
user, pw = 'katrina.mullan@mso.umt.edu', 'wvp6rau6rqb_bwy1EYT!'
url = 'https://pinnacle.azira.com/'

# start and end dates for data downloads
start_date = '2022-01-01'
end_date = date.today() - timedelta(days=7)
end_date = end_date.strftime("%Y-%m-%d")

In [19]:
end_date

'2025-12-12'

### Initialize (open page)

In [7]:
# open page - persistent across cells
p = await async_playwright().start()
browser = await p.chromium.launch(headless=False)

context = await browser.new_context(
        geolocation={"latitude": 46.8721, "longitude": -113.9940},  # Missoula-ish; pick whatever
        permissions=["geolocation"],
    )

page = await context.new_page()
#page2 = await context.new_page()

# single page version
# page = await browser.new_page()

### Navigate / execute page tests

In [8]:
await page.goto(url)

<Response url='https://pinnacle.azira.com/' request=<Request url='https://pinnacle.azira.com/' method='GET'>>

Login  |  https://pinnacle.azira.com/

In [9]:
await page.get_by_role("textbox", name="Email Address").fill(user)
await page.get_by_role("textbox", name="Password").fill(pw)
await page.get_by_role("button", name="Log in").click()

Home  |  https://pinnacle.azira.com/home/feeds/datasets

In [10]:
await page.get_by_role("button", name="Create New").click()

Format  |  https://pinnacle.azira.com/home/create/dataset

In [11]:
await page.get_by_role("button", name="Next").click()

Places |  https://pinnacle.azira.com/home/create/dataset/places

In [12]:
# click "GEoJSON/KML/ESRI"
await page.get_by_role("heading", name="GeoJSON/KML/ESRI").click()

# paste the GeoJSON data
json_path = Path('/home/vince/Documents/SmartFires/osm_fitness/New_Mexico_700k(15)/Roswell.geojson')
json_text = json_path.read_text()

await page.get_by_role("textbox", name="Enter GeoJSON").fill(json_text)

await page.get_by_role("button", name="Next").click() 

Time Frame |  https://pinnacle.azira.com/home/create/dataset/timeframe

In [13]:
await page.locator("#start-date-input").fill(start_date)
await page.locator("#end-date-input").fill(end_date)
await page.get_by_role("button", name="Next").click()



Insights 

In [ ]:
await page.get_by_text("Pin Dataset").click() 
await page.get_by_text("Common Evening and Daytime").click()   
await page.get_by_role("button", name="Next").first.click()

Details

In [15]:
city = json_path.stem
state = json_path.parent.stem.split('_')[0]
name = f"{city}_{state}"

await page.get_by_role("textbox", name="Name is required").fill(name)



In [24]:
try:
    await page.get_by_role("button", name="Submit").first.click()
    await expect(page.get_by_text("Expanded access")).to_be_visible(timeout=1000)
    print("popup")
except Exception as e:
    print("no popup")

popup


### Inspect pages

In [16]:
await page.pause()

get current url

### Teardown

In [ ]:
await browser.close()
await p.stop()

## Split a geojson into two

In [40]:
test_path = Path('/home/vince/Documents/SmartFires/osm_fitness/Utah_700k(24)/Salt Lake City_1.geojson')


In [41]:
splitJson(test_path)